In [1]:
import os
import requests

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [3]:
DATA = pd.read_excel('new_combined_activities.xlsx')
#DATA = pd.read_excel('HealthCare and Social Assistance dataset.xlsx')

In [4]:
DATA.drop_duplicates(inplace=True)

In [5]:
print('The shape of the data is ', DATA.shape)
print(DATA)

The shape of the data is  (3476, 1)
                                    Combined Activities
0          Record and review patient medical histories.
1     Feel and visually examine gums for sores and s...
2     Examine gums, using probes, to locate periodon...
3     Clean calcareous deposits, accretions, and sta...
4     Provide clinical services or health education ...
...                                                 ...
8807     Prepare reports related to compliance matters.
8808    Develop emergency response plans or procedures.
8810  Confer with organizational members to accompli...
8820  Teach safety standards or environmental compli...
8833                      Package objects for shipping.

[3476 rows x 1 columns]


In [6]:
sentences = DATA['Combined Activities'].astype(str)

combined_activity_sentences = ' '.join(sentences)
combined_activity_sentences = combined_activity_sentences.lower()
combined_activity_sentences = combined_activity_sentences.replace('\n', ' ')
print(combined_activity_sentences)

record and review patient medical histories. feel and visually examine gums for sores and signs of disease. examine gums, using probes, to locate periodontal recessed gums and signs of gum disease. clean calcareous deposits, accretions, and stains from teeth and beneath margins of gums, using dental instruments. provide clinical services or health education to improve and maintain the oral health of patients or the general public. chart conditions of decay and disease for diagnosis and treatment by dentist. expose and develop x-ray film. attend continuing education courses to maintain or update skills. apply fluorides or other cavity preventing agents to arrest dental decay. maintain dental equipment and sharpen and sterilize dental instruments. maintain patient recall system. feel lymph nodes under patient's chin to detect swelling or tenderness that could indicate presence of oral cancer. administer local anesthetic agents. remove excess cement from coronal surfaces of teeth. conduct

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
doc = nlp(combined_activity_sentences)

individual_combined_activity_sentence = [sent.text for sent in doc.sents]

individual_combined_activity_words = []
for sent in doc.sents:
    individual_combined_activity_words.extend([token.text for token in sent])

print(len(individual_combined_activity_sentence))
print("Sentences:")
print(individual_combined_activity_sentence)

print("\n")
print(len(individual_combined_activity_words))
print("Words :")
print(individual_combined_activity_words)

print("\n")
unrepeated_individual_combined_activity_words = set(individual_combined_activity_words)
unrepeated_individual_combined_activity_words = list(unrepeated_individual_combined_activity_words)
#unrepeated_individual_combined_activity_words.astype(str).tolist()
print(len(unrepeated_individual_combined_activity_words))
print('Unrepeated Words :')
print(unrepeated_individual_combined_activity_words)


3478
Sentences:
['record and review patient medical histories.', 'feel and visually examine gums for sores and signs of disease.', 'examine gums, using probes, to locate periodontal recessed gums and signs of gum disease.', 'clean calcareous deposits, accretions, and stains from teeth and beneath margins of gums, using dental instruments.', 'provide clinical services or health education to improve and maintain the oral health of patients or the general public.', 'chart conditions of decay and disease for diagnosis and treatment by dentist.', 'expose and develop x-ray film.', 'attend continuing education courses to maintain or update skills.', 'apply fluorides or other cavity preventing agents to arrest dental decay.', 'maintain dental equipment and sharpen and sterilize dental instruments.', 'maintain patient recall system.', "feel lymph nodes under patient's chin to detect swelling or tenderness that could indicate presence of oral cancer.", 'administer local anesthetic agents.', 'rem

In [9]:
lemmatized_unrepetedwords = []
for word in unrepeated_individual_combined_activity_words:
    doc = nlp(word)
    lemmatized_word = doc[0].lemma_
    lemmatized_unrepetedwords.append(lemmatized_word)

print('Lemmatized Words :')
lemmatized_unrepetaedwordswords = set(lemmatized_unrepetedwords)
lemmatized_unrepetedwords = list(lemmatized_unrepetaedwordswords)
print(lemmatized_unrepetedwords)

Lemmatized Words :
['rollaway', 'aspiration', 'observe', 'immunodeficiency', 'handicap', 'history', 'session', 'part', 'government', 'party', 'labor', 'bench', 'stick', 'comfortable', 'needle', 'within', 'tradition', 'infant', 'thermosette', 'mistake', 'taste', 'barium', 'strength', 'ultrasound', 'prior', 'defibrillation', 'invitation', 'legislative', 'line', 'regeneration', 'topic', 'architectural', 'kitchen', 'influenza', 'arm', 'expert', 'hire', 'role', 'proofread', 'notify', 'document', 'microbial', 'entry', 'attraction', 'complementary', 'ability', 'bridge', 'campus', 'assure', 'cart', 'sonogram', 'promote', 'drug', 'symptom', 'bathroom', 'tentative', 'geological', 'book', 'equal', 'interaction', 'chemical', 'picture', 'individual', 'accident', 'prop', 'leader', 'intra', 'bring', 'hall', 'lot', 'shortage', 'rewrite', 'such', 'nutrigenomic', 'tap', 'caliper', 'modality', 'create', 'mole', 'lab', 'discharge', 'candidate', 'could', 'obligation', 'card', 'affix', 'corrective', 'band',

In [10]:
SENTENCES = individual_combined_activity_sentence
WORDS = lemmatized_unrepetedwords

In [11]:
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from transformers import RobertaTokenizer, RobertaModel


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')


def get_embeddings(sentences):
    """Generates embeddings for a list of sentences using RoBERTa."""
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Load data from Excel sheet
excel_file_path = 'HealthCare and Social Assistance dataset.xlsx'
sheet_name = 'Sheet1'

# Read the Excel file using pandas
df = pd.read_excel(excel_file_path)

df['Tasks'] = df['Tasks'].fillna('')  # Replace NaN with empty strings

sentences = df['Tasks'].astype(str).tolist()  # Ensure all entries are strings

# Get embeddings for sentences
embeddings = get_embeddings(sentences)

regressor = LinearRegression()
numerical_labels = np.arange(len(sentences))  # Simulating labels based on sentence index
regressor.fit(embeddings, numerical_labels)

y_pred = regressor.predict(embeddings)

y_pred_rounded = np.round(y_pred).astype(int)

# Evaluation
mse = mean_squared_error(numerical_labels, y_pred_rounded)
r2 = r2_score(numerical_labels, y_pred_rounded)

# evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared: {r2}\n")

# interaction types corresponding to cluster labels
interaction_types = ['direct', 'indirect', 'opaque', 'automatic']

# K-means clustering on the embeddings
n_clusters = 4  # Define the number of clusters (this can be adjusted)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(embeddings)
cluster_labels = kmeans.labels_

# Map cluster labels to interaction types (just as an example)
cluster_to_interaction = {0: 'direct', 1: 'indirect', 2: 'opaque', 3: 'automatic'}

# Print clustering results for each sentence with interaction types
print("\nSentence | Cluster Interaction Type")
print("-" * 45)
for i, sentence in enumerate(sentences):
    cluster_label = cluster_labels[i]
    interaction_type = cluster_to_interaction.get(cluster_label, 'unknown')  # Map cluster to interaction type
    print(f"{sentence} | {interaction_type}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mean Squared Error (MSE): 642040.3493664339
R-squared: 0.4847769866271646


Sentence | Cluster Interaction Type
---------------------------------------------
Record and review patient medical histories. | indirect
Feel and visually examine gums for sores and signs of disease. | indirect
Examine gums, using probes, to locate periodontal recessed gums and signs of gum disease. | automatic
Clean calcareous deposits, accretions, and stains from teeth and beneath margins of gums, using dental instruments. | indirect
Provide clinical services or health education to improve and maintain the oral health of patients or the general public. | automatic
Chart conditions of decay and disease for diagnosis and treatment by dentist. | indirect
Expose and develop x-ray film. | indirect
Attend continuing education courses to maintain or update skills. | direct
Apply fluorides or other cavity preventing agents to arrest dental decay. | indirect
Maintain dental equipment and sharpen and sterilize dental 

In [28]:
# Load data from Excel sheet
excel_file_path = 'combined_table2_data_with_industry_new.xlsx'

# Read the Excel file using pandas
df = pd.read_excel(excel_file_path)

# Preprocess 'Tasks' column
df['Tasks'] = df['Tasks'].fillna('')  # Replace NaN with empty strings
sentences = df['Tasks'].astype(str).tolist()  # Ensure all entries are strings

# Get embeddings for sentences
embeddings = get_embeddings(sentences)

# Linear regression for evaluation (optional)
regressor = LinearRegression()
numerical_labels = np.arange(len(sentences))  # Simulating labels based on sentence index
regressor.fit(embeddings, numerical_labels)

y_pred = regressor.predict(embeddings)
y_pred_rounded = np.round(y_pred).astype(int)

# Define interaction types and perform K-means clustering
interaction_types = ['direct', 'indirect', 'opaque', 'automatic']
n_clusters = len(interaction_types)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(embeddings)
cluster_labels = kmeans.labels_

# Map cluster labels to interaction types
cluster_to_interaction = {i: interaction_types[i] for i in range(n_clusters)}
interaction_type_column = [cluster_to_interaction[label] for label in cluster_labels]

# Add the new Interaction Type column to the DataFrame
df['Interaction Type'] = interaction_type_column

# Define output Excel file path
output_excel_file_path = 'Final_HealthCare_Assistance_Output.xlsx'

# Write the updated DataFrame with Interaction Type column to a new Excel file
df.to_excel(output_excel_file_path, index=False)

# Print confirmation message
print(f"Final output with Interaction Type column added has been saved to '{output_excel_file_path}' successfully.")


Final output with Interaction Type column added has been saved to 'Final_HealthCare_Assistance_Output.xlsx' successfully.
